In [1]:
import torch
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric import loader

dataset = PygNodePropPredDataset(name = "ogbn-products", root = 'dataset/')
 
split_idx = dataset.get_idx_split()
data = dataset[0]

# Convert split indices to boolean masks and add them to `data`.
for key, idx in split_idx.items():
    mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    mask[idx] = True
    data[f'{key}_mask'] = mask
    

In [2]:
import random
import copy
import torch
from torch_geometric import loader
from torch_geometric.data import Data

from sklearn.decomposition import PCA

cluster_data = loader.ClusterData(data, num_parts=1000, recursive=False)

iter_num = 1
with torch.no_grad():
    for cl in cluster_data:
        print("ISOMAP UPDATES: ITERATION " + str(iter_num))
        pca = PCA(n_components=75)
        cl.x = torch.from_numpy(pca.fit_transform(cl.x))
        iter_num += 1

op_dict = {}
op_dict['embedding'] = cluster_data.data.x
op_dict['label'] = cluster_data.data.y
op_dict['train_idx'] = cluster_data.data.train_mask
op_dict['valid_idx'] = cluster_data.data.valid_mask
op_dict['test_idx'] = cluster_data.data.test_mask

torch.save(op_dict, '{}.pt'.format("PCA_EMBEDDINGS"))

Computing METIS partitioning...
Done!
